In [ ]:
%pip install python-pptx

# Automatic powerpoint generation

you want to **generate a PowerPoint file** where:

* Each slide contains **8 images** (arranged neatly),
* The **filename** (without extension) is shown **above each image**.

---

### 📌 Explanation

* **Slide layout:** 4 columns × 2 rows = 8 images per slide.
* **Spacing:** Small margins between images and text.
* **Filename:** Placed in a textbox above each image.
* **Order:** Images are inserted in sorted order by filename.

---

👉 Do you want me to **fit the images proportionally** (keeping aspect ratio, instead of forcing 2×2 inches) or is it okay to **scale them into fixed squares** as in this example?


In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pathlib import Path
from PIL import Image

def images_to_pptx(image_dir, output_pptx, images_per_slide=8):
    prs = Presentation()
    blank_slide_layout = prs.slide_layouts[6]  # empty layout

    image_dir = Path(image_dir)
    image_files = sorted(image_dir.glob("*.png"))

    # layout: 4 columns x 2 rows for 8 images
    cols = 4
    rows = 2
    margin_left = Inches(0.5)
    margin_top = Inches(1)
    slot_width = Inches(2.3)   # horizontal slot for each image
    slot_height = Inches(2.5)  # vertical slot
    fixed_height = Inches(2)   # all images scaled to this height

    for i in range(0, len(image_files), images_per_slide):
        slide = prs.slides.add_slide(blank_slide_layout)
        batch = image_files[i:i+images_per_slide]

        for j, img_path in enumerate(batch):
            row = j // cols
            col = j % cols

            # slot position
            slot_left = margin_left + col * slot_width
            slot_top = margin_top + row * slot_height
            slot_center_x = slot_left + slot_width / 2

            # compute scaled width from aspect ratio
            with Image.open(img_path) as im:
                aspect_ratio = im.width / im.height
                scaled_width = fixed_height * aspect_ratio

            # center image horizontally in slot
            left = slot_center_x - scaled_width / 2
            top = slot_top

            # add filename (centered in slot, not image width)
            textbox = slide.shapes.add_textbox(slot_left, top - Inches(0.4), slot_width, Inches(0.4))
            text_frame = textbox.text_frame
            text_frame.text = img_path.stem
            text_frame.paragraphs[0].font.size = Pt(14)

            # add image with fixed height
            slide.shapes.add_picture(str(img_path), left, top, height=fixed_height)

    prs.save(output_pptx)
    print(f"Saved presentation: {output_pptx}")


In [ ]:
# Example usage
image_dir = 'bone_originalDefect/'
output_pptx = Path(image_dir) / (Path(image_dir).stem + '_mixed.pptx')
images_per_slide=8

images_to_pptx(image_dir, output_pptx, images_per_slide)

# Two-step workflow to sort pictures

✅ You want a **two-step workflow**:

1. **Export a list of images** → one `.csv` with all filenames.
2. **Define slide placement manually** → another `.csv` where you specify which slide each image should go on.
3. **Build the PowerPoint** based on the mapping file.

---

### Step 1: Collect all image filenames

This file just lists all images in your folder:


In [ ]:
import pandas as pd
from pathlib import Path

def collect_image_list(image_dir, output_csv="images.csv"):
    image_dir = Path(image_dir)
    files = sorted(image_dir.glob("*.png"))
    df = pd.DataFrame({"filename": [f.name for f in files]})
    return df
    # df.to_csv(image_dir / output_csv, index=False)
    # print(f"Exported {len(files)} image filenames to {output_csv}")

In [ ]:
# Example usage
image_dir = 'bone_originalDefect/'

df = collect_image_list(image_dir)
df

---

### Step 2: Assign slides and create `slides.csv`

You edit this file to specify which **slide number** each image should go on:

```csv
filename,slide
subdir1.png,1
subdir2.png,1
subdir3.png,2
subdir4.png,2
```

---


In [ ]:
# Here I just auto-fill: 8 images per slide
# df["slide"] = [i // 8 + 1 for i in range(len(df))]
df["slide"] = [2, 1, 3, 1, 3, 3, 2, 3, 2, 1, 1]

# You can then adjust "slide" manually in Jupyter
display(df)

In [ ]:
# 3. Save to CSV
df.to_csv(Path(image_dir) / "slides.csv", index=False)

### Step 3: Generate PowerPoint from `slides.csv`


In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
import pandas as pd
from pathlib import Path
from PIL import Image

def build_ppt_from_csv(image_dir, slide_csv, output_pptx):
    prs = Presentation()
    blank_slide_layout = prs.slide_layouts[6]  # empty slide

    image_dir = Path(image_dir)
    df = pd.read_csv(slide_csv)

    # Group images by slide number
    grouped = df.groupby("slide")["filename"].apply(list).to_dict()

    # Grid layout: 4x2 (8 per slide)
    cols, rows = 4, 2
    margin_left, margin_top = Inches(0.5), Inches(1)
    slot_width, slot_height = Inches(2.5), Inches(2.5)  # slot "boxes"
    fixed_height = Inches(2)  # image height 

    for slide_num in sorted(grouped.keys()):
        slide = prs.slides.add_slide(blank_slide_layout)
        batch = grouped[slide_num]

        for j, fname in enumerate(batch):
            img_path = image_dir / fname
            if not img_path.exists():
                print(f"⚠️ Skipping missing file: {fname}")
                continue

            row, col = j // cols, j % cols

            # Compute slot position
            slot_left = margin_left + col * slot_width
            slot_top = margin_top + row * slot_height
            slot_center_x = slot_left + slot_width / 2

            # Open image to compute aspect ratio
            with Image.open(img_path) as im:
                aspect_ratio = im.width / im.height
                scaled_width = fixed_height * aspect_ratio

            # Center image horizontally in slot
            left = slot_center_x - scaled_width / 2
            top = slot_top

            # Add filename above image
            textbox = slide.shapes.add_textbox(left, top - Inches(0.4), scaled_width, Inches(0.4))
            text_frame = textbox.text_frame
            text_frame.text = img_path.stem
            text_frame.paragraphs[0].font.size = Pt(14)

            # Add image with fixed height, centered
            slide.shapes.add_picture(str(img_path), left, top, height=fixed_height)

    prs.save(output_pptx)
    print(f"✅ Saved presentation: {output_pptx}")


In [ ]:
# image_dir = 'bone_defect_polygon/'
slide_csv = Path(image_dir) / "slides.csv"
output_pptx = Path(image_dir) / (Path(image_dir).stem + '.pptx')

# Example usage
build_ppt_from_csv(image_dir, slide_csv, output_pptx)